# Convolutional Neural Networks with Keras (2)

#### Alternatives when using a pre-trained network:

- Just use softmax predictions for the new data. Only makes sense when new data have known classes.
- Use the bottleneck features (activations from last MaxPooling layer before fully connected layers)


Steps:

- instantiate convolutional part of the model, everything up to the fully-connected layers
- run this model on our training and test data once, recording the output (the "bottleneck features", i.e. the last activation maps before the fully-connected layers) in two numpy arrays
- train a small fully-connected model on top of the stored features

### Load VGG16

Several models, currently

- 
- 
- 

may be downloaded using <code>keras.applications</code>

In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

In [3]:
####img_width, img_height = 150, 150

top_model_weights_path = 'vgg16_weights.h5'
train_data_dir = 'data/train'
test_data_dir = 'data/test'

n_train_samples = 235
n_train_ants = 115
n_train_bees = 121

n_test_samples = 148
n_test_ants = 66
n_test_bees = 82

num_epochs = 50
batch_size = 16


In [4]:
def save_bottlebeck_features():

    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None, # no labels
        shuffle=False)
    
    bottleneck_features_train = model.predict_generator(generator, n_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'w'), bottleneck_features_train)

    generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_test = model.predict_generator(generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)


In [22]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy'))
    train_labels = np.array(
        [0] * (nb_train_samples / 2) + [1] * (nb_train_samples / 2))

    validation_data = np.load(open('bottleneck_features_validation.npy'))
    validation_labels = np.array(
        [0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)




In [109]:
save_bottlebeck_features()
train_top_model()

In [78]:
save_bottlebeck_features()
train_top_model()

In [ ]:
def score_img(img_path):
    img = load_img(img_path, grayscale=True) 
    candidate = np.expand_dims(img_to_array(img)/255, axis=0)
    pred = test_model.predict_classes(candidate, verbose=0)
    return pred[0,0]

In [ ]:
cracks_total = 837
num_cracks_found = 0
for imgfile in os.listdir('data/test/crack'):
    score = score_img('data/test/crack/' + imgfile)
    if score == 0: num_cracks_found = num_cracks_found + 1
print('Found crack in {} of {} images'.format(num_cracks_found, cracks_total))

In [ ]:
no_cracks_total = 772
num_no_cracks_found = 0
for imgfile in os.listdir('data/test/nocrack'):
    score = score_img('data/test/nocrack/' + imgfile)
    if score == 1: num_no_cracks_found = num_no_cracks_found + 1
print('Correctly said "no crack" in {} of {} images'.format(num_no_cracks_found, no_cracks_total))

In [ ]:
true_positives = num_cracks_found
false_negatives = cracks_total - true_positives
true_negatives = num_no_cracks_found
false_positives = no_cracks_total - true_negatives

accuracy = (true_positives + true_negatives) / (cracks_total + no_cracks_total)
sensitivity = true_positives / (true_positives + false_negatives)
precision = true_positives / (true_positives + false_positives)

print('Accuracy: {:.3f}, sensitivity: {:.3f}, precision: {:.3f}'.format(accuracy, sensitivity, precision))